In [8]:
# [adapted from an IPython log file]
# This notebook hopes to perform a basic test on the newly-made
# pathfinder environment.

import time; print(time.asctime())
'Fri Oct  6 00:58:37 2017'

import sys, imp
#sys.path.insert(0, '/home/usrnet/Software/installers/gym/')
sys.path.insert(0, '/home/usrnet/Work-Files/Path-Finder')
#sys.path.insert(0, '/home/usrnet/miniconda3/lib/python3.6/site-packages')
sys.path.insert(0, '/home/usrnet/Software/installers/gym/examples/agents')

import gym
from gym import envs, spaces
from gym.spaces import Discrete
from random_agent import *
from cem import *

import tensorflow as tf
import rl, keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam
from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

import cleaner

Fri Oct  6 16:32:59 2017


In [33]:
try: reload(cleaner)
except: 
    import imp, cleaner; imp.reload(cleaner)
#from cleaner import *
import cleaner; from cleaner import *

In [34]:

ex = cleaner.ExpAPI('tse2007', 'egocentric')
class MyEnv(gym.Env):
    def __init__(self, exp_env):
        self.action_space = Discrete(4)
        sz = exp_env.getGridSize()
        self.observation_space = spaces.Tuple((Discrete(sz[0]), \
                                Discrete(sz[1]), Discrete(4)))
        self.current_state = exp_env.get_random_starting_state()\
                                                    ['state']
        self.exp_env = exp_env        
        self.metadata = {'render.modes':['human','ansi','PRINT','NOPRINT']}
        self.reward_range = (0,1)
        
    def _reset(self): 
        self.current_state = self.exp_env.get_random_starting_state()['state']
        return self.current_state 
    
    def _render(self, mode=None, close=None): 
        p= print_state(self.current_state, \
                           'condensed', 'string_ret')
        if mode in ['human','PRINT']: 
            print(p)
            return p
        elif mode=='NOPRINT': 
            return p
        else: raise Exception(mode, 'render mode not defined')

    def sample(self): return random.choice(list(DVECS.keys()))

    def _step(self, actn):
        new_st, succ = self.exp_env.new_statem(\
            self.current_state, actn, valid_move_too=True)
        goalReached = self.exp_env.get_agent_loc(new_st) == \
                      self.exp_env.get_goal_loc(new_st)
        self.current_state = new_st
        return new_st, int(goalReached), (not succ) or goalReached, {}
    
   
my_env = MyEnv(ex)


-----------
----- -----
--       --
-- o    o--
--    o  --
- I       -
--  o    --
--o    @ --
--       --
----- -----
-----------



In [35]:
model = Sequential()
model.add(Flatten(input_shape = (1,11,11,4) ))
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dense(my_env.action_space.n, activation='linear'))
print(model.summary())
memory = SequentialMemory(limit=1000, window_length=1)
policy = EpsGreedyQPolicy()
dqn = DQNAgent(model=model, nb_actions = my_env.action_space.n, memory=memory, nb_steps_warmup=10, enable_dueling_network=True, dueling_type='avg', target_model_update=1e-3, policy=policy)
dqn.compile(Adam(lr=1e-5), metrics=['mae'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_7 (Flatten)          (None, 484)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 32)                15520     
_________________________________________________________________
activation_7 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_19 (Dense)             (None, 4)                 132       
Total params: 15,652
Trainable params: 15,652
Non-trainable params: 0
_________________________________________________________________
None


In [37]:
dqn.fit(my_env, nb_steps=10000, visualize=True, verbose=2)

Training for 10000 steps ...
-----------
----- -----
--   I   --
-- o    o--
--    o  --
-         -
--  o    --
--o    @ --
--       --
----- -----
-----------

-----------
-----I-----
--       --
-- o    o--
--    o  --
-         -
--  o    --
--o    @ --
--       --
----- -----
-----------

-----------
----- -----
--   I   --
-- o    o--
--    o  --
-         -
--  o    --
--o    @ --
--       --
----- -----
-----------

-----------
-----I-----
--       --
-- o    o--
--    o  --
-         -
--  o    --
--o    @ --
--       --
----- -----
-----------

-----------
----- -----
--   I   --
-- o    o--
--    o  --
-         -
--  o    --
--o    @ --
--       --
----- -----
-----------

-----------
-----I-----
--       --
-- o    o--
--    o  --
-         -
--  o    --
--o    @ --
--       --
----- -----
-----------

-----------
----- -----
--   I   --
-- o    o--
--    o  --
-         -
--  o    --
--o    @ --
--       --
----- -----
-----------

-----------
-----I-----
--       --
-- o

-----------
----- -----
--       --
-- o    o--
--    o  --
-       I -
--  o    --
--@    o --
--       --
----- -----
-----------

-----------
----- -----
--       --
-- o    o--
--    o I--
-         -
--  o    --
--@    o --
--       --
----- -----
-----------

-----------
----- -----
--      o--
-- o    I--
--    o  --
-         -
--  o    --
--@    o --
--       --
----- -----
-----------

-----------
----- -----
--      o--
-- o    I--
--    o  --
-         -
--  o    --
--@    o --
--       --
----- -----
-----------

  108/10000: episode: 5, duration: 0.029s, episode steps: 6, steps per second: 209, episode reward: 0.000, mean reward: 0.000 [0.000, 0.000], mean action: 1.333 [0.000, 3.000], mean observation: 0.157 [0.000, 1.000], loss: 0.009111, mean_absolute_error: 0.085599, mean_q: 0.036600
-----------
----- -----
--       --
-- @    o--
--    o  --
-       I -
--  o    --
--o    o --
--       --
----- -----
-----------

-----------
----- -----
--       --
-- @    o--
--    

-----------
-----I-----
--       --
-- @    o--
--    o  --
-         -
--  o    --
--o    o --
--       --
----- -----
-----------

-----------
----- -----
--   I   --
-- @    o--
--    o  --
-         -
--  o    --
--o    o --
--       --
----- -----
-----------

-----------
-----I-----
--       --
-- @    o--
--    o  --
-         -
--  o    --
--o    o --
--       --
----- -----
-----------

-----------
----- -----
--   I   --
-- @    o--
--    o  --
-         -
--  o    --
--o    o --
--       --
----- -----
-----------

-----------
-----I-----
--       --
-- @    o--
--    o  --
-         -
--  o    --
--o    o --
--       --
----- -----
-----------

-----------
----- -----
--   I   --
-- @    o--
--    o  --
-         -
--  o    --
--o    o --
--       --
----- -----
-----------

-----------
----- -----
--       --
-- @ I  o--
--    o  --
-         -
--  o    --
--o    o --
--       --
----- -----
-----------

-----------
----- -----
--       --
-- @  I o--
--    o  --
-        

-----------
----- -----
--   I   --
-- o    o--
--    @  --
-         -
--  o    --
--o    o --
--       --
----- -----
-----------

-----------
-----I-----
--       --
-- o    o--
--    @  --
-         -
--  o    --
--o    o --
--       --
----- -----
-----------

-----------
----- -----
--   I   --
-- o    o--
--    @  --
-         -
--  o    --
--o    o --
--       --
----- -----
-----------

-----------
-----I-----
--       --
-- o    o--
--    @  --
-         -
--  o    --
--o    o --
--       --
----- -----
-----------

-----------
----- -----
--   I   --
-- o    o--
--    @  --
-         -
--  o    --
--o    o --
--       --
----- -----
-----------

-----------
-----I-----
--       --
-- o    o--
--    @  --
-         -
--  o    --
--o    o --
--       --
----- -----
-----------

-----------
----- -----
--   I   --
-- o    o--
--    @  --
-         -
--  o    --
--o    o --
--       --
----- -----
-----------

-----------
-----I-----
--       --
-- o    o--
--    @  --
-        

-----------
-----I-----
--       --
-- o    o--
--    @  --
-         -
--  o    --
--o    o --
--       --
----- -----
-----------

  401/10000: episode: 21, duration: 0.308s, episode steps: 79, steps per second: 257, episode reward: 0.000, mean reward: 0.000 [0.000, 0.000], mean action: 0.582 [0.000, 3.000], mean observation: 0.157 [0.000, 1.000], loss: 0.005147, mean_absolute_error: 0.087809, mean_q: 0.043010
-----------
----- -----
--       --
-- o    o--
--    @  --
-       I -
--  o    --
--o    o --
--       --
----- -----
-----------

-----------
----- -----
--       --
-- o    o--
--    @  --
-         -
--  o   I--
--o    o --
--       --
----- -----
-----------

-----------
----- -----
--       --
-- o    o--
--    @  --
-       I -
--  o    --
--o    o --
--       --
----- -----
-----------

-----------
----- -----
--       --
-- o    o--
--    @ I--
-         -
--  o    --
--o    o --
--       --
----- -----
-----------

-----------
----- -----
--       --
-- o    o--
--  

...

-----------
----- -----
--       --
-- o    o--
--    @  --
-   I     -
--       --
--o o  o --
--       --
----- -----
-----------

-----------
----- -----
--       --
-- o    o--
--  I @  --
-         -
--       --
--o o  o --
--       --
----- -----
-----------

-----------
----- -----
--       --
-- oI   o--
--    @  --
-         -
--       --
--o o  o --
--       --
----- -----
-----------

-----------
----- -----
--       --
--oI    o--
--    @  --
-         -
--       --
--o o  o --
--       --
----- -----
-----------

-----------
----- -----
-- I     --
--o     o--
--    @  --
-         -
--       --
--o o  o --
--       --
----- -----
-----------

-----------
----- -----
-- I     --
--o     o--
--    @  --
-         -
--       --
--o o  o --
--       --
----- -----
-----------

 9562/10000: episode: 730, duration: 0.049s, episode steps: 12, steps per second: 243, episode reward: 0.000, mean reward: 0.000 [0.000, 0.000], mean action: 0.917 [0.000, 3.000], mean observation: 0.15

-----------
----- -----
--       --
-- o  I o--
--    o  --
-         -
--  o    --
--o    @ --
--       --
----- -----
-----------

-----------
----- -----
--       --
-- o    o--
--    I  --
-     o   -
--  o    --
--o    @ --
--       --
----- -----
-----------

-----------
----- -----
--       --
-- o    o--
--       --
-     I   -
--  o o  --
--o    @ --
--       --
----- -----
-----------

-----------
----- -----
--       --
-- o    o--
--    I  --
-         -
--  o o  --
--o    @ --
--       --
----- -----
-----------

-----------
----- -----
--       --
-- o  I o--
--       --
-         -
--  o o  --
--o    @ --
--       --
----- -----
-----------

-----------
----- -----
--       --
-- o   Io--
--       --
-         -
--  o o  --
--o    @ --
--       --
----- -----
-----------

-----------
----- -----
--     I --
-- o    o--
--       --
-         -
--  o o  --
--o    @ --
--       --
----- -----
-----------

-----------
----- -----
--     I --
-- o    o--
--       --
-        

-----------
----- -----
--       --
-- o    o--
--    o  --
-         -
--  o    --
--o    @ --
--  I    --
----- -----
-----------

-----------
----- -----
--       --
-- o    o--
--    o  --
-         -
--  o    --
--o    @ --
--  I    --
----- -----
-----------

 9759/10000: episode: 745, duration: 0.015s, episode steps: 3, steps per second: 198, episode reward: 0.000, mean reward: 0.000 [0.000, 0.000], mean action: 1.333 [0.000, 3.000], mean observation: 0.157 [0.000, 1.000], loss: 0.004276, mean_absolute_error: 0.041295, mean_q: 0.075655
-----------
----- -----
--       --
-- o    o--
--    o  --
-        I-
--  @    --
--o    o --
--       --
----- -----
-----------

 9760/10000: episode: 746, duration: 0.008s, episode steps: 1, steps per second: 133, episode reward: 0.000, mean reward: 0.000 [0.000, 0.000], mean action: 2.000 [2.000, 2.000], mean observation: 0.157 [0.000, 1.000], loss: 0.001452, mean_absolute_error: 0.046979, mean_q: 0.079636
-----------
----- -----
--       --

-----------
----- -----
--       --
-- o    o--
--    @  --
-         -
--  I    --
--o o  o --
--       --
----- -----
-----------

-----------
----- -----
--       --
-- o    o--
--    @  --
-   I     -
--       --
--o o  o --
--       --
----- -----
-----------

-----------
----- -----
--       --
-- o    o--
--  I @  --
-         -
--       --
--o o  o --
--       --
----- -----
-----------

-----------
----- -----
--       --
-- oI   o--
--    @  --
-         -
--       --
--o o  o --
--       --
----- -----
-----------

-----------
----- -----
--  I    --
-- o    o--
--    @  --
-         -
--       --
--o o  o --
--       --
----- -----
-----------

-----------
----- -----
--  I    --
-- o    o--
--    @  --
-         -
--       --
--o o  o --
--       --
----- -----
-----------

 9864/10000: episode: 751, duration: 0.053s, episode steps: 13, steps per second: 246, episode reward: 0.000, mean reward: 0.000 [0.000, 0.000], mean action: 1.000 [0.000, 3.000], mean observation: 0.15

-----------
----- -----
--       --
-- oI   o--
--    o  --
-         -
--       --
--o o  @ --
--       --
----- -----
-----------

-----------
----- -----
--       --
-- o    o--
--  I o  --
-         -
--       --
--o o  @ --
--       --
----- -----
-----------

-----------
----- -----
--       --
-- oI   o--
--    o  --
-         -
--       --
--o o  @ --
--       --
----- -----
-----------

-----------
----- -----
--       --
-- o    o--
--  I o  --
-         -
--       --
--o o  @ --
--       --
----- -----
-----------

-----------
----- -----
--       --
-- oI   o--
--    o  --
-         -
--       --
--o o  @ --
--       --
----- -----
-----------

-----------
----- -----
--       --
-- o    o--
--  I o  --
-         -
--       --
--o o  @ --
--       --
----- -----
-----------

-----------
----- -----
--       --
-- oI   o--
--    o  --
-         -
--       --
--o o  @ --
--       --
----- -----
-----------

-----------
----- -----
--       --
-- o    o--
--  I o  --
-        